# S07_SparkSQL

In [1]:
import pyspark
print(pyspark.__version__)

3.5.3


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sepomex").getOrCreate()

locatelTiempo = "locateltiempo.cvs"
df = spark.read.csv(locatelTiempo, header = True, inferSchema = True, sep="|")
df.printSchema()


from pyspark.sql.types import StructType, StructField, StringType, IntegerType
 
schema = StructType([
    StructField("d_codigo", StringType(), True),
    StructField("d_asenta", StringType(), True),
    StructField("d_tipo_asenta", StringType(), True),
    StructField("D_mnpio", StringType(), True),
    StructField("d_estado", StringType(), True),
    StructField("d_ciudad", StringType(), True),
    StructField("d_CP", StringType(), True),
    StructField("c_estado", StringType(), True),
    StructField("c_oficina", StringType(), True),
    StructField("c_CP", StringType(), True),
    StructField("c_tipo_asenta", StringType(), True),
    StructField("c_mnpio", StringType(), True),
    StructField("id_asenta_cpcons", StringType(), True),
    StructField("d_zona", StringType(), True),
    StructField("c_cve_ciudad", StringType(), True)
])

sepomex_df = spark.read.option("header","true")\
    .option("delimiter","|")\
    .option("skipRows","1")\
    .option("encoding","UTF-8")\
    .schema(schema)\
    .csv("sepomex.csv")
sepomex_df.count()

root
 |-- 2024-01-30,2024-10-19T09:17:32.000-06:00,QUEJA DE TRANSPORTE PUBLICO,FEMENINO,,CERRADO,NA,NA,,: string (nullable = true)



113147

In [7]:
strql = """SELECT
MIN(edad) AS min_edad, MAX(edad) AS max_edad, AVG(edad) avg_edad,
STDDEV(edad) as stddev_edad, COUNT(edad) AS nr
PERCERTILE_CONT(0.5) WITH GROUP(ORDER BY edad) AS mediana_edad
FROM locatel
"""

spark.sql(strql).show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'PERCERTILE_CONT'.(line 4, pos 0)

== SQL ==
SELECT
MIN(edad) AS min_edad, MAX(edad) AS max_edad, AVG(edad) avg_edad,
STDDEV(edad) as stddev_edad, COUNT(edad) AS nr
PERCERTILE_CONT(0.5) WITH GROUP(ORDER BY edad) AS mediana_edad
^^^
FROM locatel


In [ ]:
spark.sql("SELECT COUNT(*) FROM LOCATEL")

In [ ]:
strql = """SELECT edad, COUNT(*) nr 
FROM locatel 
GROUP BY edad 
ORDER BY edad DESC"""

spark.sql(strql).show(200)

In [ ]:
from pyspark.sql.functions import col, when, avg

edad_promedio = df.filter(col("edad") != 16090.0).agg(avg("edad")).first()[0]
edad_promedio

In [ ]:
#Reemplazar el valor por edad promedio
df = df.withColumn(
    "edad", when(col("edad") == 16090.0, edad_promedio).otherwise(col("edad"))
)

df.createOrReplaceTempView("locatel")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df_edad = spark.sql("SELECT edad FROM locatel WHERE edad IS NOT NULL")
#.toPandas()

plt.figure(figsize=(8,6))
sns.boxplot(df_edad('edad'),orient='h')

plt.title("Boxplot de edad")
plt.xlabel("Edad")

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=df_edad,x="edad",kde=True)
plt.xlabel("Edad")
plt.ylabel("Frecuencia")
plt.show()